In [4]:
!pip install transformers

     |████████████████████████████████| 1.8MB 7.8MB/s 
     |████████████████████████████████| 890kB 41.3MB/s 
     |████████████████████████████████| 3.2MB 33.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=3bac6110a4e308fb3fdaf447117f090cf5a026d33caa93217053a7975b25d72c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
!pip install torch===1.7.1 torchvision===0.8.2 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 12.8MB 45.5MB/s 
     |████████████████████████████████| 7.6MB 52.9MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [6]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Importing and preprocessing datasets

In [7]:
pos_df = pd.read_csv('positive.csv', delimiter=';', header=None)
neg_df = pd.read_csv('negative.csv', delimiter=';', header=None)

In [8]:
pos_df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
5,408906761769598976,1386325943,JustinB94262583,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1,0,0,0,40,6,16,0
6,408906762436481024,1386325943,twinkleAYO,"RT @SpoonLamer: Ох,900 :D ну это конечно же @t...",1,0,1,0,5169,58,43,2
7,408906764114206720,1386325944,pycalyruhog,RT @veregijytaqo: У тебя есть ухажёр? Нет - мо...,1,0,2,0,393,112,153,0
8,408906764608749568,1386325944,grishintv,Поприветствуем моего нового читателя @Alexey17...,1,0,0,0,5872,1387,1431,12
9,408906765841870848,1386325944,alina_612,Теперь у меня есть частичка Сиднея :) #Sydney ...,1,0,0,0,263,5,36,0


In [9]:
df = pd.concat([pos_df, neg_df], ignore_index=True).sample(frac=1).reset_index(drop=True)
df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11
0,410299413069062144,1386657977,shocker_padonok,@antonusov @mavroid будем знать что тебе дарит...,1,0,0,0,2950,84,64,3
1,415571179433238528,1387914864,Fom4enkov,@romandope сегодня обновилось приложение и выл...,-1,0,0,0,2886,239,96,2
2,416669263706214400,1388176668,pinkunicorn_RIP,@frerard_mcr69 опять твой сарказм(((9999(((999,-1,0,0,0,7946,202,185,1
3,410469934833995776,1386698633,swagswagonyou_,ЛИНТА СКИНЬ МНИ ПОШЛЫЙ ФФ С ДЖА)на ночь почита...,1,0,0,0,34047,1548,1880,22
4,410645922478297088,1386740592,katya_gayda,"проспала, но приехала на 20 мин. раньше , надо...",1,0,0,0,3836,23,22,1
5,423082749042638848,1389705762,N_muller25,"Пака меня не было, все забыли(\n#жизньболь",-1,0,0,0,7956,315,304,2
6,416216974520098817,1388068834,kodilazimul,Подпевать всяким скримо группам во время уборк...,-1,0,0,0,10,0,5,0
7,409840159719710720,1386548483,pemotyxubev,день идет удивительно хорошо — у меня все сего...,1,0,0,0,466,162,160,0
8,422997402199793664,1389685414,lifehell14,Фааааак....... Сегодня контроша по литре -_- С...,-1,0,0,0,39,16,44,0
9,410793561794162690,1386775791,Vik_Blond,@tigrrrrrrrrrrr я не буду переубеждать..пусть ...,1,0,0,0,11315,81,40,0


In [10]:
df = df[df.columns[3:5]]
df.columns = [0,1]
df.head(10)

,0,1
0,@antonusov @mavroid будем знать что тебе дарит...,1
1,@romandope сегодня обновилось приложение и выл...,-1
2,@frerard_mcr69 опять твой сарказм(((9999(((999,-1
3,ЛИНТА СКИНЬ МНИ ПОШЛЫЙ ФФ С ДЖА)на ночь почита...,1
4,"проспала, но приехала на 20 мин. раньше , надо...",1
5,"Пака меня не было, все забыли(\n#жизньболь",-1
6,Подпевать всяким скримо группам во время уборк...,-1
7,день идет удивительно хорошо — у меня все сего...,1
8,Фааааак....... Сегодня контроша по литре -_- С...,-1
9,@tigrrrrrrrrrrr я не буду переубеждать..пусть ...,1


In [11]:
df.shape

(226834, 2)

In [12]:
import re

def preprocess1(text):
        """Minimal text preprocessing
        """
        text = re.sub("^\s+|([A-Za-z])+|(@[A-Za-z0-9\_\-]+)|\n|\r|\s+$", ' ', text)
        return text.strip()


def preprocess2(num):
    if num == -1:
        return 0
    else:
        return num

In [13]:
df[0] = df.apply(lambda x: preprocess1(x[0]), axis=1)
df[1] = df.apply(lambda y: preprocess2(y[1]), axis=1)

In [14]:
df.head(10)

,0,1
0,будем знать что тебе дарить на следующее дэрэ ...,1
1,сегодня обновилось приложение и вылетает при а...,0
2,опять твой сарказм(((9999(((999,0
3,ЛИНТА СКИНЬ МНИ ПОШЛЫЙ ФФ С ДЖА)на ночь почита...,1
4,"проспала, но приехала на 20 мин. раньше , надо...",1
5,"Пака меня не было, все забыли( #жизньболь",0
6,Подпевать всяким скримо группам во время уборк...,0
7,день идет удивительно хорошо — у меня все сего...,1
8,Фааааак....... Сегодня контроша по литре -_- С...,0
9,я не буду переубеждать..пусть ты останешься пр...,1


In [15]:
df[1].value_counts()

1    114911
0    111923
Name: 1, dtype: int64

In [16]:
df = df[:2000]
df[1].value_counts()

1    1019
0     981
Name: 1, dtype: int64

In [17]:
df

,0,1
0,будем знать что тебе дарить на следующее дэрэ ...,1
1,сегодня обновилось приложение и вылетает при а...,0
2,опять твой сарказм(((9999(((999,0
3,ЛИНТА СКИНЬ МНИ ПОШЛЫЙ ФФ С ДЖА)на ночь почита...,1
4,"проспала, но приехала на 20 мин. раньше , надо...",1
...,...,...
1995,"мне не повезло. было очень холодно, подхватила...",0
1996,#Хам хочет как обычно кинуть весь народ! Хочет...,0
1997,На улице снег и на рабочем столе снежинки пада...,1
1998,: Даже очень :(,0


# Loading model

In [18]:
model_name = "DeepPavlov/rubert-base-cased-conversational"

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Preparing the Dataset

In [19]:
tokenized = df[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [20]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [21]:
np.array(padded).shape

(2000, 70)

In [22]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 70)

## Train

In [23]:
input_ids = torch.tensor(padded).to(torch.int64)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [24]:
features = last_hidden_states[0][:,0,:].numpy()  
"""Вот тут на выходе последнего слоя я получаю, что у тензора размерность 2, хотя по сути, она должна быть равной 70"""

IndexError: ignored

In [ ]:
labels = df[1]

# Model #2: Train

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

In [ ]:
lr_clf.score(test_features, test_labels)